In [1]:
%load_ext autoreload
%autoreload 2


>> #### Import required modules

In [2]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from kneed import KneeLocator
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

Ignoring Warnings

In [3]:
import warnings
warnings.filterwarnings('ignore')


#### Import modules

In [4]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from preprocessing import PreProcess
from overview import Overview
from plot import Plot
from logger import Logger

In [5]:
# Initialize logger
logger = Logger("userSatisfaction.log").get_app_logger()
logger.info("Initialized user satisfaction logger")

#### Import the Data

In [6]:
# df = pd.read_excel('../data/Week1_challenge_data_source.xlsx')
df_org = pd.read_csv('../data/cleaned_data_outliers.csv')
df = df_org.copy()

>> ### Satisfaction Analysis


In [19]:
# Import user experience data
user_exp = pd.read_csv('../data/user_exp.csv')

# Import user engagement data
user_eng = pd.read_csv('../data/user_eng.csv')

In [20]:
#  Drop the Unnamed:0 from user_eng
user_exp. drop('Unnamed: 0', axis=1, inplace=True)
user_exp.head()

,cluster,avg_rtt_total,avg_tp_total,total_avg_tcp_total
0,1,0.741217,0.664495,0.095095
1,0,0.396618,0.909921,0.121397
2,2,0.568755,0.815210,0.109322
3,2,0.605140,0.793439,0.065272
4,0,0.285623,0.955171,0.077894


Engagement Score calculation

AttributeError: 'NoneType' object has no attribute 'head'